In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script> so much code! click <a href="javascript:code_toggle()">here</a>.''')

In [4]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import pandas as pd
import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc
import pickle
import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy

#from matplotlib import reload
import arrow
import gsw
import time

%matplotlib inline

plt.rcParams.update({'font.size': 20,
                     'xtick.labelsize' : 20,
                     'ytick.labelsize' : 20})



In [5]:

import BC_helper_methods as hm
import importlib
importlib.reload(hm)

<module 'BC_helper_methods' from '/ocean/tjarniko/MEOPAR/analysis_tereza/notebooks/RIVER_PAPER/future_pilot/BC_helper_methods.py'>

In [6]:
# def co2_from_year(scen,tyear):
#     import pandas as pd
#     import numpy as np

#     #get witnessed co2, currently only takes 2-4.5 and 5-8.5

#     df = pd.read_csv('./meinshausen/meinshausen_scenario_conc.csv')
#     data_top = list(df.columns) 
# #     print('available data columns')
# #     print(data_top)

#     year = np.array(df['year'][:])
#     # SSP1_1pt9_NH = np.array(df['SSP1-1.9_NH'][:])
#     # SSP1_2pt6_NH = np.array(df['SSP1-2.6_NH'][:])
#     # SSP2_4pt5_NH = np.array(df['SSP2-4.5'][:])
#     SSP2_4pt5_NH = np.array(df['SSP2-4.5'][:])
#     SSP5_8pt5_NH = np.array(df['SSP5-8.5'][:])

#     q = np.shape(tyear)
#     tco2_ar = np.zeros_like(tyear)
#     tyear_r = np.ravel(tyear)
#     tco2_ar_r = np.ravel(tco2_ar)
#     for i in range(0,len(tco2_ar_r)):
#         # building for 2_4pt5
#         if (scen == '2_4pt5'): 

#             tco2_ar_r[i] = SSP2_4pt5_NH[year == (int(tyear_r[i]))]

#         if scen == '5_8pt5':

#             tco2_ar_r[i] = SSP5_8pt5_NH[year == (int(tyear_r[i]))]
    
#     #tco2_ar = tco2_ar_r.reshape(q[0],q[1])

#     return tco2_ar

In [11]:
print('warlurs')

warlurs


In [15]:
obs_year = [2015]
target_year = [2050]
scen = '2_4pt5'
DIC = 1600
TA = 2190
T = 7
S = 31


def Pacella_TCO2(obs_year,target_year, scen, tsal, ttemp, dic, ta, verbose = False):
    

    watermass_witnessed_co2_obs = int(hm.co2_from_year_pointmeas(scen,obs_year))
    watermass_witnessed_co2_target = int(hm.co2_from_year_pointmeas(scen,target_year))
    
    if verbose:
        print(f'obs year: {obs_year}')
        print(f'obs year co2: {watermass_witnessed_co2_obs}')
        print(f'target year: {target_year}')
        print(f'target year co2: {watermass_witnessed_co2_target}')
    
    DIC_obs_eq = hm.find_DIC_corresp_to_pco2(tsal, ttemp, watermass_witnessed_co2_obs, ta, 1, 0)
    del_TCO2 = dic - DIC_obs_eq
    
    DIC_target_eq = hm.find_DIC_corresp_to_pco2(tsal, ttemp, watermass_witnessed_co2_target, ta, 1, 0)
    DIC_target_final = DIC_target_eq + del_TCO2
    DIC_intrusion = DIC_target_final-dic
    #DIC_obs_real = hm.find_DIC_corresp_to_pco2(tsal, ttemp, watermass_witnessed_co2_obs, tta, 1, 0)
    if verbose:
        print('')
        #print(del_TCO2)
        print(f'this is what DIC would be at equilibrium (obs): {DIC_obs_eq}')
        print(f'this is what DIC actually is (obs): {dic}')
        print(f'this is the del TCO2: {del_TCO2}')
        print(f'this is what DIC would be at equilibrium (target): {DIC_target_eq}')
        print(f'this is what DIC actually should/would be (target): {DIC_target_eq + del_TCO2}')
        print(f'this is the intrusion: {DIC_intrusion}')
        
    #print(f'this is the intrusion: {DIC_intrusion}')
    return DIC_intrusion

Pacella_TCO2(obs_year,target_year, scen, S, T, DIC, TA, verbose = True)



obs year: [2015]
obs year co2: 401
target year: [2050]
target year co2: 509

this is what DIC would be at equilibrium (obs): 2055.4799999995957
this is what DIC actually is (obs): 1600
this is the del TCO2: -455.47999999959575
this is what DIC would be at equilibrium (target): 2088.0599999995866
this is what DIC actually should/would be (target): 1632.5799999999908
this is the intrusion: 32.57999999999083


32.57999999999083

In [8]:
def calc_surface_intrusion_Pacella_method(arrowdate, obs_year, target_year, scen):
    import numpy as np
    import netCDF4 as nc
    import gsw
    
    test_LO = hm.load_nc(arrowdate)
    
    tdate = arrowdate
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}'
    #open dataset & retrieve relevant variables, calculate potential density

    #get just the surface intrusion
    zlevels = (test_LO['deptht'][:])
    sal = test_LO['vosaline'][0,0,0,:]
    temp = test_LO['votemper'][0,0,0,:]
    sigma0 = gsw.sigma0(sal,temp)
    DIC = test_LO['DIC'][0,0,0,:]
    TA = test_LO['TA'][0,0,0,:]
    
    intrusion = np.zeros_like(DIC)
    start = time.time()

    print(np.shape(DIC))
    #hardcode size of array 
    for i in range(0,950):
        if i%100 == 0:
            print(i)
        intrusion[i] = Pacella_TCO2(obs_year, target_year, scen, sal[i], temp[i], DIC[i], TA[i], verbose = False)
    print('seconds taken at the hard part')
    print(time.time()-start)
    
    
    f = nc.Dataset(f'./JdF_future_DIC/LO_TY_{target_year}_scen_{scen}_{ymd}_DIC_PacellaSurf.nc','w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('preindustrial_DIC')
    g.createDimension('xval', 950)
    g.createDimension('depth', 1)

    ts = g.createVariable('DIC_intrusion','f4',('depth','xval'))
    ts[:] = intrusion

    f.close()

In [9]:
# start ='2017-01-01'
# end ='2017-12-31'

# start_run = arrow.get(start)
# end_run = arrow.get(end)

# arrow_array = []

# for r in arrow.Arrow.span_range('day', start_run, end_run):
#     arrow_array.append(r)
    
    
# for i in range(0,1):
#     calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')

In [10]:
from multiprocessing import Process

start ='2017-01-01'
end ='2017-12-31'

start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)
    
    
def func1():
  print('func1: starting')
  for i in range(0,60):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')

def func2():
  print('func2: starting')
  for i in range(60,120):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')
      
def func3():
  print('func3: starting')
  for i in range(120,180):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')
        
def func4():
  print('func4: starting')
  for i in range(180,240):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')
        
def func5():
  print('func5: starting')
  for i in range(240,300):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')
        
def func6():
  print('func36: starting')
  for i in range(300,366):
        calc_surface_intrusion_Pacella_method(arrow_array[i][0], 2017, 2050, '2_4pt5')
        
if __name__ == '__main__':
  p1 = Process(target=func1)
  p1.start()
  p2 = Process(target=func2)
  p2.start()
  p3 = Process(target=func3)
  p3.start()
  p4 = Process(target=func4)
  p4.start()
  p5 = Process(target=func5)
  p5.start()
  p6 = Process(target=func6)
  p6.start()                
    
  p1.join()
  p2.join()
  p3.join()
  p4.join()
  p5.join()
  p6.join()

func1: starting
2017-01-01T00:00:00+00:00
func2: starting
2017-03-02T00:00:00+00:00
func3: starting
2017-05-01T00:00:00+00:00
func4: starting
2017-06-30T00:00:00+00:00
func5: starting
2017-08-29T00:00:00+00:00
func36: starting
2017-10-28T00:00:00+00:00
(950,)
0
(950,)
0
(950,)
0
(950,)
0
(950,)
0
(950,)
0
100
100
100
100
100
100
200
200
200
200
200
200
300
300
300
300
300
300
400
400
400
400
400
400
500
500
500
500
500
500
600
600
600
600
600
600
700
700
700
700
700
700
800
800
800
800
800
800


Process Process-3:
Process Process-5:
Process Process-4:
Process Process-6:
Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tjarniko/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
 

KeyboardInterrupt: 

In [ ]:
def deltaDIC_method()

In [ ]:
w = nc.Dataset('./JdF_future_DIC/LO_TY_2050_scen_2_4pt5_y2017m01d01_DIC_nosurfnodeep.nc')
print(w['preindustrial_DIC'])

In [ ]:
tcmap = cm.cm.deep
cmap = tcmap
test_LO = nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_y2018m01d01.nc')
zlevels = (test_LO['deptht'][:])

fig = plt.figure(figsize=(11,3.5))

ax = fig.add_subplot(1,5,1)
section_slice = np.arange(0, 95)
sd = (w['preindustrial_DIC']['sigma0'][:,0:95])
tplt = np.ma.masked_values(sd,0)
mesh = ax.pcolormesh(section_slice[:],zlevels[:],tplt[:,:], cmap = tcmap)
cbar = fig.colorbar(mesh, ax=ax)
ax.invert_yaxis()
plt.title('sigma0') 
ax.set_ylabel('depth')
ax.set_xticklabels([])

ax = fig.add_subplot(1,5,2)
section_slice = np.arange(0, 95)
sd = (w['preindustrial_DIC']['water_age'][:,0:95])
tplt = np.ma.masked_values(sd,0)
mesh = ax.pcolormesh(section_slice[:],zlevels[:],tplt[:,:], cmap = tcmap)
cbar = fig.colorbar(mesh, ax=ax)
ax.invert_yaxis()
plt.title('water_age') 
ax.set_ylabel('depth')
ax.set_xticklabels([])


ax = fig.add_subplot(1,5,3)
section_slice = np.arange(0, 95)
sd = (w['preindustrial_DIC']['AOU_stoich'][:,0:95])
tplt = np.ma.masked_values(sd,0)
mesh = ax.pcolormesh(section_slice[:],zlevels[:],tplt[:,:], cmap = tcmap)
cbar = fig.colorbar(mesh, ax=ax)
ax.invert_yaxis()
plt.title('stoichiometric AOU') 
ax.set_ylabel('depth')
ax.set_xticklabels([])

ax = fig.add_subplot(1,5,4)
section_slice = np.arange(0, 95)
sd = (w['preindustrial_DIC']['final_target_DIC'][:,0:95])
tplt = np.ma.masked_values(sd,0)
mesh = ax.pcolormesh(section_slice[:],zlevels[:],tplt[:,:], cmap = tcmap)
cbar = fig.colorbar(mesh, ax=ax)
ax.invert_yaxis()
plt.title('final target DIC') 
ax.set_ylabel('depth')
ax.set_xticklabels([])


ax = fig.add_subplot(1,5,5)
section_slice = np.arange(0, 95)
sd = (w['preindustrial_DIC']['targetyear_pref_dic'][:,0:95]-\
     w['preindustrial_DIC']['obsyear_pref_dic'][:,0:95])
tplt = np.ma.masked_values(sd,0)
mesh = ax.pcolormesh(section_slice[:],zlevels[:],tplt[:,:], cmap = tcmap)
cbar = fig.colorbar(mesh, ax=ax)
ax.invert_yaxis()
plt.title('DIC intrusion yr 2050') 
ax.set_ylabel('depth')
ax.set_xticklabels([])
        
plt.suptitle('y2015m01d01 - basic parameters and extra DIC, year 2050, SSP245 ', fontsize = 15)
plt.tight_layout()

In [ ]:
q = w['preindustrial_DIC']['AOU_stoich'][:,0:95]
print(q)